In [1]:
import importlib
import torch
import torch.nn as nn
from torch import optim

import sys

In [2]:
# local imports 
sys.path.append("..")

import transformer_model.model as model
importlib.reload(model)

import transformer_model.model_utils as model_utils
importlib.reload(model_utils)

import transformer_model.data_utils as data_utils
importlib.reload(data_utils)

from data import ArticlesIter, articles_filepath

ImportError: cannot import name 'ArticlesIter' from 'data' (/home/cstainsby/class/dataProj/bonion/src/notebooks/../data/__init__.py)

## Setup

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Load Vocab

In [ ]:
SAVE_PATH = "../transformer_model/vocab_save/" + "embeddings_vocab.pt"

emb_vocab_obj = torch.load(SAVE_PATH)

print("Vocab size:", len(emb_vocab_obj))

### Load Model

In [ ]:
SAVE_PATH = "../transformer_model/model_save/" + "glove_emb_model.pt"

emb_model = torch.load(SAVE_PATH)

print("Loaded hyperparameters:")
print("---------------------------")
print("Model Dimension:    ", emb_model.d_model)
print("Num Layers:         ", emb_model.num_layers)
print("Num Heads:          ", emb_model.nhead)
print("Max Length:         ", emb_model.max_seq_len)
print("Dropout Per.:       ", emb_model.dropout)

## Test Text Generation
Given our mappings we just read in we can now have the transformer make some text.

In [ ]:
start_str = "I went to the"
number_of_words = 10

gen_words = start_str.split(" ")
        
print(start_str, end=" ")
        
for word in model_utils.generate_text(
    model=emb_model,
    vocab=emb_vocab_obj,
    start_str=start_str,
    max_length=number_of_words):
    
    gen_words.append(word)
    print(word, end=" ")

## Train the Model

In [ ]:
learning_rate = 3e-4

optimizer = optim.Adam(emb_model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

The data will be split into 1000 batches. For each batch, the content will be pulled out. This content will be tokenized and fed into the *train_model* function.

In [ ]:
article_iter = ArticlesIter(batch_size=1000)
total_loss = 0.0
total_batches = 0

for batch_i, batch in enumerate(article_iter):
    if batch_i < 5:
        print("Training batch:", batch_i)
        batch_content = batch["Content"]
        print(batch_content)

        # for article in batch_content:
        #     tokenized_batch_content = tokenize_raw_data(article, emb_vocab_obj)

        #     print(article)
        #     print(tokenized_batch_content)

        total_loss += model_utils.train_batch(
            model=emb_model,
            vocab=emb_vocab_obj,
            optimizer=optimizer,
            criterion=criterion,
            batch=batch_content,
            device=device
        )
        total_batches += 1

print("Average Loss:", total_loss/total_batches)


### Save the trained model
Save the model to a .pt file